<a href="https://colab.research.google.com/github/Harshini0614/CSM/blob/master/Sleeping_Disorders_Detection_Outliers_Removal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'sleep-health-and-lifestyle-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3321433%2F6491929%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240423%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240423T183650Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D40ed0e9151f42bb052adcc765fad26fc86844382df5c7fb51656f77826102512ca63f4a0c73a9983840b99ff6a24ffbdfd4ee5350185077d66409f154f8f779c0a54d5449071a6d1e8e520a23b24561f1eeb579ac0666e0d54bfe1dc26c16b328466f781227b5f128084740039a9de5737ebc9ab50ccb43b0eafbe50124f8b7d1409ee986eee89fd949274a2fd8cadecdcf3a4ac2f6d5a07520f4eb6be9e85a712f0633d06c0646364cbf4d90998a2b350e7893210ec466c447431a2fc1005cdac5c42c2006ccc0ad7a2f99927a96f81e32a933347e1458bcd2b7b3fe702749b5e1c6b7bdde24bb44ec8d7edc94efeb106f43757f5d988e266d49d86567bb2f7'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 2604 bytes downloaded
Downloaded and uncompressed: sleep-health-and-lifestyle-dataset
Data source import complete.


# Introduction

Sleep is a critical factor in maintaining overall health and well-being, yet various lifestyle choices can significantly influence the quality and duration of sleep. This analysis delves into the intricate relationship between lifestyle factors and sleep health, providing valuable insights for individuals seeking to enhance their sleep patterns. As a data scientist, deciphering these connections allows for informed decision-making, ultimately optimizing sleep health.

# Dataset Columns

The dataset includes the following columns:

* Person ID: An identifier for each individual in the dataset.

* Gender: The gender of the person (Male/Female).

* Age: The age of the person in years.

* Occupation: The occupation or profession of the person.

* Sleep Duration (hours): The number of hours the person sleeps per day.

* Quality of Sleep (scale: 1-10): A subjective rating of sleep quality (1 to 10).

* Physical Activity Level (minutes/day): The daily minutes of physical activity.

* Stress Level (scale: 1-10): A subjective rating of stress level (1 to 10).

* BMI Category: The BMI category of the person (Underweight, Normal, Overweight).

* Blood Pressure (systolic/diastolic): The blood pressure measurement (systolic over diastolic).

* Heart Rate (bpm): Resting heart rate in beats per minute.

* Daily Steps: The number of steps taken per day.

* Sleep Disorder: The presence or absence of a sleep disorder (None, Insomnia, Sleep Apnea).



# Objective:
Identify and analyze the presence or absence of sleep disorders, including Insomnia and Sleep Apnea

# Importing Libraries:

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder ,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Loading the Dataset:

In [3]:
data=pd.read_csv('/kaggle/input/sleep-health-and-lifestyle-dataset/Sleep_health_and_lifestyle_dataset.csv')

# Exploratory Data Analysis (EDA):

In [4]:
data.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           155 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB


In [6]:
data.duplicated().sum()


0

In [7]:
data.isna().sum()

Person ID                    0
Gender                       0
Age                          0
Occupation                   0
Sleep Duration               0
Quality of Sleep             0
Physical Activity Level      0
Stress Level                 0
BMI Category                 0
Blood Pressure               0
Heart Rate                   0
Daily Steps                  0
Sleep Disorder             219
dtype: int64

In [8]:
data.dropna(inplace=True)

# Data Preprocessing:

Label Encoding: Transform categorical variables into numerical representations for machine learning models.

In [9]:
label_encoder=LabelEncoder()

In [10]:
cat_cols=['Gender','Occupation','BMI Category','Sleep Disorder']
for col in cat_cols:
    data[col] = label_encoder.fit_transform(data[col])

In [11]:
data

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
3,4,1,28,5,5.9,4,30,8,2,140/90,85,3000,1
4,5,1,28,5,5.9,4,30,8,2,140/90,85,3000,1
5,6,1,28,8,5.9,4,30,8,2,140/90,85,3000,0
6,7,1,29,9,6.3,6,40,7,2,140/90,82,3500,0
16,17,0,29,4,6.5,5,40,7,1,132/87,80,4000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,0,59,4,8.1,9,75,3,3,140/95,68,7000,1
370,371,0,59,4,8.0,9,75,3,3,140/95,68,7000,1
371,372,0,59,4,8.1,9,75,3,3,140/95,68,7000,1
372,373,0,59,4,8.1,9,75,3,3,140/95,68,7000,1


In [12]:

 # Split the 'Blood Pressure' column into two columns
data[['Systolic BP', 'Diastolic BP']] = data['Blood Pressure'].str.split('/', expand=True)

# Convert the new columns to numeric type
data[['Systolic BP', 'Diastolic BP']] = data[['Systolic BP', 'Diastolic BP']].apply(pd.to_numeric)

# Drop the original 'Blood Pressure' column
data = data.drop('Blood Pressure', axis=1)


In [13]:
data

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Sleep Disorder,Systolic BP,Diastolic BP
3,4,1,28,5,5.9,4,30,8,2,85,3000,1,140,90
4,5,1,28,5,5.9,4,30,8,2,85,3000,1,140,90
5,6,1,28,8,5.9,4,30,8,2,85,3000,0,140,90
6,7,1,29,9,6.3,6,40,7,2,82,3500,0,140,90
16,17,0,29,4,6.5,5,40,7,1,80,4000,1,132,87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,0,59,4,8.1,9,75,3,3,68,7000,1,140,95
370,371,0,59,4,8.0,9,75,3,3,68,7000,1,140,95
371,372,0,59,4,8.1,9,75,3,3,68,7000,1,140,95
372,373,0,59,4,8.1,9,75,3,3,68,7000,1,140,95


In [14]:
data.drop('Person ID', axis=1, inplace=True)

# Splitting the Data:

In [15]:
# Splitting the data into features (X) and the target variable (y)
X = data.drop('Sleep Disorder', axis=1)  # Features
y = data['Sleep Disorder']  # Target variable

# Splitting the data into training and testing sets (e.g., 80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Scaling Data: Standardizing numerical features like 'Age', 'Sleep Duration', and 'Daily Steps' through scaling is essential.
This process ensures that variables with different units and scales contribute equally to the model, preventing biases and allowing machine learning algorithms to effectively learn patterns, leading to improved model performance.


In [16]:
# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the scaler on the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the testing data using the same scaler
X_test_scaled = scaler.transform(X_test)

# Modeling with Lazy Predict:

Lazy Predict: Employing Lazy Predict allows for a quick and comprehensive comparison of multiple machine learning models without the need for extensive manual tuning. This streamlined approach aids in the efficient identification of promising models, saving time during the initial stages of analysis and providing valuable insights into the dataset's predictability.


In [17]:
pip install lazypredict

In [18]:
from lazypredict.Supervised import LazyClassifier
# Use LazyClassifier
clf = LazyClassifier()
models, predictions = clf.fit(X_train_scaled, X_test_scaled, y_train, y_test)

# Print the model performance
print(models)

100%|██████████| 29/29 [00:01<00:00, 28.95it/s]

[LightGBM] [Info] Number of positive: 61, number of negative: 63
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000306 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 100
[LightGBM] [Info] Number of data points in the train set: 124, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.491935 -> initscore=-0.032261
[LightGBM] [Info] Start training from score -0.032261
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

# Let's try to imporve the accuracy by handling the outliers

In [19]:
num_col = ['Age', 'Sleep Duration', 'Quality of Sleep', 'Physical Activity Level', 'Stress Level',
           'Heart Rate', 'Daily Steps', 'Systolic BP', 'Diastolic BP']

Q1 = data[num_col].quantile(0.25)
Q3 = data[num_col].quantile(0.75)
IQR = Q3 - Q1

data = data[~((data[num_col] < (Q1 - 1.5 * IQR)) | (data[num_col] > (Q3 + 1.5 * IQR))).any(axis=1)]

In [20]:
# Splitting the data into features (X) and the target variable (y)
X = data.drop('Sleep Disorder', axis=1)  # Features
y = data['Sleep Disorder']  # Target variable

# Splitting the data into training and testing sets (e.g., 80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [21]:

# Fit and transform the scaler on the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the testing data using the same scaler
X_test_scaled = scaler.transform(X_test)

In [22]:
models, predictions = clf.fit(X_train_scaled, X_test_scaled, y_train, y_test)

# Print the model performance
print(models)

'tuple' object has no attribute '__name__'
Invalid Classifier(s)


100%|██████████| 29/29 [00:00<00:00, 39.21it/s]

[LightGBM] [Info] Number of positive: 32, number of negative: 55
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000037 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 73
[LightGBM] [Info] Number of data points in the train set: 87, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.367816 -> initscore=-0.541597
[LightGBM] [Info] Start training from score -0.541597
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

**After effectively addressing outliers in the dataset, we observed a notable improvement in model accuracy. Outlier handling has played a crucial role in enhancing the robustness of our predictive models, resulting in more accurate and reliable predictions.**

# ****Your feedback and support are highly valuable – if you found this notebook helpful, consider giving it an upvote.****